In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar
from core.s3 import S3AssetManager


In [2]:
notebook_name = "adipell_sultana"
s3 = S3AssetManager(notebook_name=notebook_name)

In [ ]:
base_path = "raw/honduras"
granel = s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='sin_adipel_alien_engorde')
con = s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='con_adipel_alien_engorde')
con_punto = s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='con_adipel_punto')
sultana =  s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='la_sultana_con')


Reading Excel from: s3://galileo-c4e9a2f1/raw/honduras/Pruebas Adipell Honduras y Salvador.xlsx
Reading Excel from: s3://galileo-c4e9a2f1/raw/honduras/Pruebas Adipell Honduras y Salvador.xlsx
Reading Excel from: s3://galileo-c4e9a2f1/raw/honduras/Pruebas Adipell Honduras y Salvador.xlsx
Reading Excel from: s3://galileo-c4e9a2f1/raw/honduras/Pruebas Adipell Honduras y Salvador.xlsx


In [5]:
sultana_group = sultana.groupby(["producto"]).agg(
    rendimiento_promedio=('Ton / h', "mean"),
    consumo_energia_promedio=('Consumo Energético kWh/T', "mean"),
    carga_promedio=("Carga", "mean"),
    presion_vapor_promedio=("Presión vapor (bar)", "mean"),
    humedad_entrada_promedio=('H harina (H entrada)', "mean"),
    humedad_acondicionador_promedio=('H harina acondicionada (Humedad salida)', "mean"),
    humedad_producto_terminado_promedio=('H producto terminado', "mean"),
    temperatura_harina_promedio=('T harina', 'mean'),
    temperatura_acondicionador_promedio=('T acondicionamiento', 'mean'),
    temperatura_peletizadora_promedio=('T pelletizadora', 'mean'),
    pdi_promedio=('PDI', 'mean'),
    finos_promedio=('finos de pellets', 'mean'),
    finos_menor_1mm_cumbre_1=('finos < 1mm crumble 1', 'mean'),
    finos_menor_1mm_cumbre_2=('finos < 1mm crumble 2', 'mean'),
    finos_mayor_1mm_menor_3_35mm_cumbre_2=('finos >1mm y < 3.35mm crumble 1', 'mean'),
    finos_mayor_3_35mm_cumbre_1=("finos > 3.35mm crumble 1", "mean"),
    finos_mayor_3_35mm_cumbre_2=("finos > 3.35mm crumble 2", "mean"),
).reset_index()

cols_finos = ['finos_promedio', 'finos_menor_1mm_cumbre_1', 'finos_menor_1mm_cumbre_2','humedad_entrada_promedio',
              'humedad_acondicionador_promedio', 'humedad_producto_terminado_promedio', 'finos_mayor_1mm_menor_3_35mm_cumbre_2', 'finos_mayor_3_35mm_cumbre_1', 'finos_mayor_3_35mm_cumbre_2']
for col in cols_finos:
    sultana_group[col] = sultana_group[col]*100
sultana_group["experimento"] = "Exp III"
sultana_group

,producto,rendimiento_promedio,consumo_energia_promedio,carga_promedio,presion_vapor_promedio,humedad_entrada_promedio,humedad_acondicionador_promedio,humedad_producto_terminado_promedio,temperatura_harina_promedio,temperatura_acondicionador_promedio,temperatura_peletizadora_promedio,pdi_promedio,finos_promedio,finos_menor_1mm_cumbre_1,finos_menor_1mm_cumbre_2,finos_mayor_1mm_menor_3_35mm_cumbre_2,finos_mayor_3_35mm_cumbre_1,finos_mayor_3_35mm_cumbre_2,experimento
0,con adipell,17.62,11.370,0.7280,1.7,12.881667,14.7150,12.5350,31.7,81.833333,82.166667,91.581667,2.548333,18.196667,16.751667,61.7650,20.038333,20.836667,Exp III
1,sin adipell,17.20,11.575,0.7175,1.7,12.910000,15.1275,12.6125,33.2,82.500000,82.000000,92.070000,1.587500,17.862500,16.387500,60.1225,22.132500,23.172500,Exp III


In [7]:
fig_pdi =plot_bar(
df=sultana_group,
x_col="experimento",
y_col="pdi_promedio",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='delta',
#title="PDI (%)",
)
fig_pdi.show()
s3.save_plotly_html(fig_pdi, "pdi_sultana.html")

In [8]:
fig_rendimiento =plot_bar(
df=sultana_group,
x_col="experimento",
y_col="rendimiento_promedio",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='pct',
#title="PDI (%)",
)
fig_rendimiento.show()
s3.save_plotly_html(fig_rendimiento, "rendimiento_promedio_sultana.html")

In [9]:
fig_finos =plot_bar(
df=sultana_group,
x_col="experimento",
y_col="finos_promedio",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='delta',
#title="PDI (%)",
)
fig_finos.show()
s3.save_plotly_html(fig_finos, "finos_promedio_sultana.html")

In [10]:
fig_final = create_subplot_grid(
    figures=[fig_rendimiento, fig_pdi, fig_finos],
    rows=1, cols=3,
    titles=["Rendimiento (Ton/H)", "PDI (%)", "Finos (%)"],
    main_title="Métricas de rendimiento y calidad",
    shared_y=False,
    width=1000, height=400
)

fig_final.show()
s3.save_plotly_html(fig_final, "pdi_finos_rendimiento_sultana.html")

In [11]:
fig_consumo_energia_promedio =plot_bar(
df=sultana_group,
x_col="experimento",
y_col="consumo_energia_promedio",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='pct',
#title="PDI (%)",
)
fig_consumo_energia_promedio.show()
s3.save_plotly_html(fig_consumo_energia_promedio, "consumo_energia_promedio_sultana.html")

In [12]:
fig_carga_promedio =plot_bar(
df=sultana_group,
x_col="experimento",
y_col="carga_promedio",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='pct',
#title="PDI (%)",
)
fig_carga_promedio  .show()
s3.save_plotly_html(fig_carga_promedio, "carga_promedio_sultana.html")

In [13]:
fig_presion_vapor_promedio =plot_bar(
df=sultana_group,
x_col="experimento",
y_col="presion_vapor_promedio",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='pct',
#title="PDI (%)",
)
fig_presion_vapor_promedio.show()
s3.save_plotly_html(fig_presion_vapor_promedio, "presion_vapor_promedio_sultana.html")

In [14]:
fig_final = create_subplot_grid(
    figures=[fig_consumo_energia_promedio, fig_carga_promedio, fig_presion_vapor_promedio],
    rows=1, cols=3,
    titles=["Consumo Energético Promedio (kWh/T)",'Carga Promedio (%)',  "Presión vapor Promedio (bar)"],
    main_title="Métricas de rendimiento y calidad",
    shared_y=False,
    width=1000, height=400
)

fig_final.show()
s3.save_plotly_html(fig_final, "consumo_energia_carga_sultana.html")

In [17]:
sultana_group

,producto,rendimiento_promedio,consumo_energia_promedio,carga_promedio,presion_vapor_promedio,humedad_entrada_promedio,humedad_acondicionador_promedio,humedad_producto_terminado_promedio,temperatura_harina_promedio,temperatura_acondicionador_promedio,temperatura_peletizadora_promedio,pdi_promedio,finos_promedio,finos_menor_1mm_cumbre_1,finos_menor_1mm_cumbre_2,finos_mayor_1mm_menor_3_35mm_cumbre_2,finos_mayor_3_35mm_cumbre_1,finos_mayor_3_35mm_cumbre_2
0,con adipell,17.62,11.370,0.7280,1.7,12.881667,14.7150,12.5350,31.7,81.833333,82.166667,91.581667,2.548333,18.196667,16.751667,61.7650,20.038333,20.836667
1,sin adipell,17.20,11.575,0.7175,1.7,12.910000,15.1275,12.6125,33.2,82.500000,82.000000,92.070000,1.587500,17.862500,16.387500,60.1225,22.132500,23.172500


In [17]:
import plotly.graph_objects as go

def grafico_lineas_generalizado(
    df,
    x_cols,
    x_labels,
    filename=None,
    title="Comparación de Temperaturas",
    y_title="Temperatura (°C)",
    series_col="producto",
    series_names=("con adipell", "sin adipell"),
    colors=("#1C8074", "#666666")
):
    """
    Genera un gráfico de líneas con puntos en forma de X para múltiples series.

    Parámetros:
    -----------
    df : DataFrame con los datos.
    x_cols : list[str] columnas del DataFrame para el eje X.
    x_labels : list[str] etiquetas del eje X (mismo orden que x_cols).
    filename : str, opcional, si se pasa guarda el gráfico en HTML.
    title : str, título del gráfico.
    y_title : str, etiqueta del eje Y.
    series_col : str, columna con los nombres de las series (ej. 'producto').
    series_names : tuple[str], nombres de las series a graficar.
    colors : tuple[str], colores para las series.
    """
    fig = go.Figure()

    for serie, color in zip(series_names, colors):
        valores = df.loc[df[series_col] == serie, x_cols].values.flatten()
        fig.add_trace(go.Scatter(
            x=x_labels,
            y=valores,
            mode="lines+markers+text",
            name=serie,
            line=dict(color=color, width=2),
            marker=dict(symbol="x", size=10, color=color),  # Puntos en forma de X
            text=[f"{v:.1f}" for v in valores],
            textposition="top center"
        ))

    # Estilo general
    fig.update_layout(
        title=title,
        xaxis_title="Etapa del proceso",
        yaxis_title=y_title,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font=dict(color="black"),
        showlegend=True,
        margin=dict(t=80, r=40, b=40, l=60)
    )

    if filename:
        fig.write_html(filename, include_plotlyjs="cdn", full_html=True)

    return fig


In [18]:
x_cols = [
    "temperatura_harina_promedio",
    "temperatura_acondicionador_promedio",
    "temperatura_peletizadora_promedio"
]
x_labels = ["Harina", "Acondicionador", "Peletizadora"]

f = grafico_lineas_generalizado(

    sultana_group,
    title="Comparación de Temperaturas por punto de medida",

    x_cols=x_cols,
    x_labels=x_labels,)
f.show()
s3.save_plotly_html(f, 'lineas_temperaturas.html')

In [19]:
x_cols = [
    "humedad_entrada_promedio",
    "humedad_acondicionador_promedio",
    "humedad_producto_terminado_promedio"
]
x_labels = ["Humedad Entrada", "Humedad Acondicionador", "Humedad Producto Terminado"]

f = grafico_lineas_generalizado(
    sultana_group,
    title="Comparación de Humedades por punto de medida",
    y_title="Humedad (%)",
    x_cols=x_cols,
    x_labels=x_labels,
)
f.show()
s3.save_plotly_html(f, 'lineas_humedades.html')